# Newsletter Product Metadata Extraction

**Objective:** This notebook aims to extract structured product metadata (product names, categories, prices, URLs, etc.) from HTML newsletter content. It leverages the Google Generative AI SDK (Gemini API) for parsing the HTML and understanding its content, and then stores the extracted information into a Pandas DataFrame.

**Steps:**
1.  **Setup & Configuration:** Import libraries, load API keys, configure the Google Generative AI SDK, and define file paths.
2.  **Helper Functions:** Define functions to read HTML files.
3.  **Gemini API Interaction:** Define a function to send HTML content to the Gemini API (via the Python SDK) with a specific schema to get structured JSON output.
4.  **Main Processing:** Orchestrate the reading of newsletter files, calling the Gemini API for each, and collecting results.
5.  **Execution & Results:** Run the extraction process, display the resulting DataFrame, and save it to a CSV file.

**Prerequisites:**
- Ensure you have the `google-generativeai`, `pandas`, and `python-dotenv` libraries installed:
  `pip install google-generativeai pandas python-dotenv`
- A `.env` file in the project root containing your `GEMINI_API_KEY`.
- HTML newsletter files located in the `data/raw/` directory.

In [1]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
import asyncio
import time # For adding delays

# For running async code in Jupyter environments smoothly
import nest_asyncio
nest_asyncio.apply()

# Google Generative AI SDK
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold # For safety settings if needed
from google.api_core import exceptions as google_api_exceptions # For specific API errors

# Check if running in Canvas/Pyodide for informational purposes (less critical for SDK usage than direct fetch)
try:
    from js import fetch # fetch itself is not used for API calls now, but js global indicates environment
    IS_CANVAS_ENV = True
    print("Running in a Canvas-like (Pyodide/JavaScript) environment.")
except ImportError:
    IS_CANVAS_ENV = False
    print("Not running in a Canvas-like environment.")

Not running in a Canvas-like environment.


## 2. Configuration

Set up essential configuration variables here.
-   **API Key:** Loaded from a `.env` file and used to configure the Google Generative AI SDK.
-   **Directories:** Define paths for raw data input and processed data output.
-   **File Information:** List of newsletter HTML files to process.
-   **API Parameters:** Define limits and delays for API interaction.

In [12]:
# --- API Key Loading ---
def load_api_key():
    """Loads the Gemini API key from the .env file."""
    # Assuming .env file is in the PROJECT_ROOT, adjust if it's elsewhere relative to the notebook
    # For example, if .env is in PROJECT_ROOT and this notebook is in PROJECT_ROOT/notebooks:
    # dotenv_path = os.path.join(os.path.dirname(os.getcwd()), '.env') 
    # load_dotenv(dotenv_path=dotenv_path)
    # Or, more simply, if load_dotenv() is called after PROJECT_ROOT is correctly set:
    # load_dotenv(dotenv_path=os.path.join(PROJECT_ROOT, '.env'))

    # For now, assume .env is discoverable by load_dotenv() from the execution context or project root.
    # If running the notebook from the 'notebooks' subdir, and .env is in parent (project root):
    project_root_for_env = os.path.abspath(os.path.join(os.getcwd(), ".."))
    env_path = os.path.join(project_root_for_env, '.env')
    if os.path.exists(env_path):
        load_dotenv(dotenv_path=env_path)
        print(f".env file loaded from: {env_path}")
    else:
        # Fallback to default search path of load_dotenv (current dir, then parents)
        load_dotenv() 
        print(f"Attempting to load .env from default search paths (e.g., current dir: {os.getcwd()}).")


    api_key = os.getenv("GEMINI_API_KEY")
    if not api_key:
        print("Warning: GEMINI_API_KEY not found in .env file.")
        if IS_CANVAS_ENV:
            print("In Canvas, if API_KEY is intended to be injected, the SDK might need specific handling or may not work if it expects a non-empty key for configuration.")
            return "" # Return empty string as per previous logic for Canvas. SDK behavior with this needs testing.
        else:
            print("API calls will likely fail without an API key for the google-generativeai SDK.")
            return None
    print("GEMINI_API_KEY loaded.") # Removed "from .env file" as path might vary
    return api_key

# --- Determine Project Root ---
# If os.getcwd() is the 'notebooks' directory as stated by the user.
current_working_dir = os.getcwd()
print(f"Current working directory (os.getcwd()): {current_working_dir}")
if os.path.basename(current_working_dir).lower() == "notebooks":
    PROJECT_ROOT = os.path.abspath(os.path.join(current_working_dir, ".."))
    print(f"Detected 'notebooks' subdirectory. Setting PROJECT_ROOT to: {PROJECT_ROOT}")
else:
    PROJECT_ROOT = current_working_dir # Assume running from project root if not in 'notebooks'
    print(f"Not in 'notebooks' subdirectory. Assuming PROJECT_ROOT is: {PROJECT_ROOT}")


API_KEY = load_api_key() # Call after PROJECT_ROOT is determined for correct .env pathing if needed


# --- Configure Google Generative AI SDK ---
SDK_CONFIGURED_SUCCESSFULLY = False
if API_KEY: # If a key string is present (not None, not empty)
    try:
        genai.configure(api_key=API_KEY)
        SDK_CONFIGURED_SUCCESSFULLY = True
        print("Google Generative AI SDK configured successfully.")
    except Exception as e:
        print(f"Error configuring Google Generative AI SDK with loaded API key: {e}")
elif IS_CANVAS_ENV and API_KEY == "":
    print("Canvas environment: API_KEY is an empty string. The google-generativeai SDK typically requires a valid API key string for genai.configure().")
    print("If the Canvas environment injects credentials in a way the SDK can pick up by default (e.g., ADC), it might work. Otherwise, explicit configuration with a valid key is needed.")
else:
    print("Google Generative AI SDK not configured due to missing or empty API key (and not in a Canvas-like scenario with an empty key placeholder).")


# --- Directory and File Paths (now relative to the correctly determined PROJECT_ROOT) ---
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
DATA_RAW_DIR = os.path.join(DATA_DIR, "raw")
DATA_PROCESSED_DIR = os.path.join(DATA_DIR, "processed")
OUTPUT_CSV_FILENAME = "extracted_newsletter_product_metadata_sdk.csv" 
OUTPUT_CSV_PATH = os.path.join(DATA_PROCESSED_DIR, OUTPUT_CSV_FILENAME)

# --- Newsletter Files ---
newsletter_files_info = [
    {"name": "konvy", "path": os.path.join(DATA_RAW_DIR, "konvy.html")},
    {"name": "gramedia", "path": os.path.join(DATA_RAW_DIR, "gramedia.html")},
    {"name": "uniqlo", "path": os.path.join(DATA_RAW_DIR, "uniqlo.html")},
]

# --- API Interaction Parameters ---
HTML_CONTENT_LIMIT = 18000  
API_CALL_DELAY_SECONDS = 7 
API_MAX_RETRIES = 3        
API_RETRY_DELAY_SECONDS = 10 

# --- Ensure directories exist ---
if not os.path.exists(DATA_RAW_DIR): os.makedirs(DATA_RAW_DIR); print(f"Created directory: {DATA_RAW_DIR}")
if not os.path.exists(DATA_PROCESSED_DIR): os.makedirs(DATA_PROCESSED_DIR); print(f"Created directory: {DATA_PROCESSED_DIR}")

print(f"Final Project Root used for paths: {PROJECT_ROOT}")
print(f"Raw Data Directory used: {DATA_RAW_DIR}")
print(f"Processed Data Directory used: {DATA_PROCESSED_DIR}")
print(f"Output CSV will be: {OUTPUT_CSV_PATH}")

Current working directory (os.getcwd()): /mnt/c/Users/tduricic/Development/workspace/conversational-reco/notebooks
Detected 'notebooks' subdirectory. Setting PROJECT_ROOT to: /mnt/c/Users/tduricic/Development/workspace/conversational-reco
.env file loaded from: /mnt/c/Users/tduricic/Development/workspace/conversational-reco/.env
GEMINI_API_KEY loaded.
Google Generative AI SDK configured successfully.
Final Project Root used for paths: /mnt/c/Users/tduricic/Development/workspace/conversational-reco
Raw Data Directory used: /mnt/c/Users/tduricic/Development/workspace/conversational-reco/data/raw
Processed Data Directory used: /mnt/c/Users/tduricic/Development/workspace/conversational-reco/data/processed
Output CSV will be: /mnt/c/Users/tduricic/Development/workspace/conversational-reco/data/processed/extracted_newsletter_product_metadata_sdk.csv


## 3. Helper Function: Read HTML File

This function reads the content of an HTML file. It includes basic error handling for file operations and a fallback to create dummy files with example content if the specified files are not found. **For actual use, ensure your real HTML files are in `data/raw/`.**

In [13]:
def read_html_file(file_path, source_name_for_dummy_content="unknown"):
    """
    Reads HTML content from a file.
    Includes a fallback to create/use dummy content if the file is not found or empty.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        if not content.strip(): 
            print(f"Warning: File at {file_path} is empty.")
            raise FileNotFoundError 
        return content
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}. Using or creating dummy content for '{source_name_for_dummy_content}'.")
        dummy_html_map = {
            "konvy": """<div class="milled-email-body"> <span class="mcnPreviewText">7.7 คุ้มสุดที่นี่! 77฿ Shockprice</span> <a href="https://www.konvy.com/interpharma/interpharma-multivitamin-20-tablets-66890.html" title="Interpharma Multivitamin 20 Tablets"> <img src="https://images.milledcdn.com/2022-07-04/XQ3vCbw8LhHNe72i/juyVbA1kUnCC.png" alt="Interpharma Multivitamin"> Interpharma Multivitamin 20 Tablets </a> Price: 390฿. Original Price: 450฿. Discount: Save 60฿. Category: Health Supplement. Availability: Limited Stock. </div>""",
            "gramedia": """<div id="milled-emaildiv"> <a href="https://www.gramedia.com/products/mice-cartoon--telekomunikasi-mengubah-peradaban"> <img src="https://images.milledcdn.com/2025-05-19/Cjb4VmzDPX1RHsiO/T9Y4xg8zPthP.avif" alt="Mice Cartoon"> Mice Cartoon: Telekomunikasi Mengubah Peradaban </a> Price: Rp 80.000. Category: Comic. <a href="https://www.gramedia.com/products/one-piece-108"> <img src="https://images.milledcdn.com/2025-05-19/Cjb4VmzDPX1RHsiO/rNmV7KQ6urHi.avif" alt="One Piece 108"> One Piece 108 </a> Price: Rp 45.000. Category: Manga. </div>""",
            "uniqlo": """<div id="milled-emaildiv"> <a href="https://www.uniqlo.com/us/en/products/E465185-000/00"> <img src="https://images.milledcdn.com/2025-05-18/hvqmJGxcgjt0TjPX/zC7S2sysZ9Gl.png" alt="AIRism T-Shirt"> AIRism Cotton Oversized T-Shirt </a> Price: $14.90. Promotion: Limited time offer. Category: T-Shirt. <a href="https://www.uniqlo.com/us/en/products/E465755-000/00"> <img src="https://images.milledcdn.com/2025-05-18/hvqmJGxcgjt0TjPX/3NW25MasLyQx.png" alt="Women AIRism T-Shirt"> Women AIRism Cotton T-Shirt </a> Price: $19.90. Category: T-Shirt. </div>"""
        }
        dummy_content = dummy_html_map.get(source_name_for_dummy_content, f"<html><body><p>Default dummy content for {source_name_for_dummy_content}</p></body></html>")
        try:
            with open(file_path, 'w', encoding='utf-8') as f: f.write(dummy_content)
            print(f"A dummy file with sample content has been created at {file_path}. Please replace it with your actual newsletter HTML.")
            return dummy_content
        except Exception as e_write:
            print(f"Could not write dummy file to {file_path}: {e_write}")
            return None
    except Exception as e:
        print(f"An unexpected error occurred while reading {file_path}: {e}")
        return None

## 4. Gemini API Interaction: `extract_metadata_with_gemini`

This core function takes HTML content and sends it to the Gemini API using the `google-generativeai` Python SDK.
-   It uses a predefined **JSON schema** to instruct Gemini on the desired output format for product metadata.
-   A detailed **prompt** guides Gemini on what to extract.
-   It includes **retry logic** for API calls.
-   The SDK handles the underlying HTTP requests.

In [14]:
async def extract_metadata_with_gemini(html_content, source_name):
    """
    Extracts structured product metadata from HTML content using the Google Generative AI SDK.
    Assumes genai SDK is already configured.
    """
    if not SDK_CONFIGURED_SUCCESSFULLY and not (IS_CANVAS_ENV and API_KEY == ""): # If not configured and not in a state where Canvas *might* handle it
        print(f"Error: Google Generative AI SDK not configured. Skipping API call for {source_name}.")
        return []
    if not html_content:
        print(f"HTML content for {source_name} is empty. Skipping extraction.")
        return []

    print(f"Starting metadata extraction for {source_name} using Google Generative AI SDK...")

    schema = {
        "type": "ARRAY",
        "items": {
            "type": "OBJECT",
            "properties": {
                "product_name": {"type": "STRING", "description": "Name of the product. Extract from text or image alt text. If not found, use null."},
                "category": {"type": "STRING", "description": "Category of the product (e.g., 'T-shirt', 'Comic Book', 'Skincare'). Infer if not explicit. If not found, use null."},
                "price": {"type": "STRING", "description": "Current price of the product, including currency symbol. If not found, use null."},
                "original_price": {"type": "STRING", "description": "Original price if a discount is mentioned. If not found, use null."},
                "discount": {"type": "STRING", "description": "Discount information. If not found, use null."},
                "availability": {"type": "STRING", "description": "Availability status (e.g., 'In Stock', 'Limited Time Offer'). If not found, use null."},
                "product_url": {"type": "STRING", "description": "Direct URL to the product page. This is a key field. If not found, use null."},
                "image_url": {"type": "STRING", "description": "URL of the main product image. If not found, use null."},
                "promotion_description": {"type": "STRING", "description": "Concise description of any promotion. If not found, use null."}
            },
            "required": ["product_name", "product_url"]
        }
    }

    prompt_text = f"""
    You are an expert HTML newsletter parser. Your task is to extract structured product metadata from the following HTML content.
    The HTML content is from a newsletter by '{source_name}'.
    Focus on identifying individual products or promotional items and their details.
    Extract information for each distinct product or item.

    Please extract the following information for each product/item found, adhering to the types specified in the schema:
    - product_name: (STRING) The name or title of the product/item.
    - category: (STRING) The general category of the product.
    - price: (STRING) The selling price. Include currency.
    - original_price: (STRING) The original price if a sale/discount is indicated.
    - discount: (STRING) Any discount percentage or amount.
    - availability: (STRING) Information like 'Limited Time Offer'.
    - product_url: (STRING) The hyperlink (URL) for the product. This is very important.
    - image_url: (STRING) The source URL of the product image.
    - promotion_description: (STRING) A brief description of the promotion.

    If information is not available, use JSON null. Ensure the output is a JSON array of objects strictly adhering to the schema.
    Exclude general navigation, social media, or unsubscribe links unless they directly promote a product offer.
    Prioritize items that are actual products for sale or specific promotions.

    HTML Content (first {HTML_CONTENT_LIMIT} characters):
    ```html
    {html_content[:HTML_CONTENT_LIMIT]}
    ```
    Return a valid JSON array of objects as per the schema.
    """
    # Safety settings can be adjusted if needed
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    }

    model = genai.GenerativeModel(
        model_name='gemini-1.5-flash-latest', # Using the specified model
        safety_settings=safety_settings 
    )
    
    generation_config_obj = genai.types.GenerationConfig(
        response_mime_type="application/json",
        response_schema=schema
    )

    response_text_for_error_reporting = ""

    for attempt in range(API_MAX_RETRIES):
        try:
            print(f"Attempt {attempt + 1} of {API_MAX_RETRIES} for {source_name} with SDK...")
            response = await model.generate_content_async(
                contents=[prompt_text], # Pass prompt as part of contents list
                generation_config=generation_config_obj
            )
            
            response_text_for_error_reporting = response.text # This should be the JSON string
            extracted_data = json.loads(response_text_for_error_reporting) # This should be an array
            
            print(f"Successfully extracted {len(extracted_data)} items for {source_name} using SDK.")
            for item in extracted_data: # Add source for tracking
                item['newsletter_source'] = source_name
            return extracted_data

        except json.JSONDecodeError as e:
            print(f"SDK Error decoding JSON response for {source_name} (Attempt {attempt + 1}): {e}")
            print(f"Problematic text from SDK: {response_text_for_error_reporting[:500]}...")
            if attempt < API_MAX_RETRIES - 1:
                print(f"Retrying in {API_RETRY_DELAY_SECONDS} seconds...")
                await asyncio.sleep(API_RETRY_DELAY_SECONDS)
            else:
                print(f"Max retries reached for {source_name}. Failed to decode JSON from SDK.")
                return []
        except google_api_exceptions.ResourceExhausted as e:
            print(f"SDK API Rate Limit Error for {source_name} (Attempt {attempt + 1}): {e}")
            wait_time = API_RETRY_DELAY_SECONDS * (attempt + 2) # Exponential backoff
            print(f"Rate limited. Waiting for {wait_time} seconds...")
            await asyncio.sleep(wait_time)
        except genai.types.BlockedPromptException as e:
            print(f"SDK Prompt Blocked Error for {source_name} (Attempt {attempt + 1}): {e}")
            print("The prompt was blocked by safety settings. This is not retryable with the same prompt.")
            return [] # Not retryable
        except genai.types.StopCandidateException as e:
            print(f"SDK Stop Candidate Error for {source_name} (Attempt {attempt + 1}): {e}")
            print(f"Content generation stopped. Reason: {e.finish_reason}. This might indicate an issue with the content or schema.")
            # Could inspect e.candidates for more details if needed
            return [] # Likely not retryable without changes
        except Exception as e:
            print(f"An unexpected SDK error occurred for {source_name} (Attempt {attempt + 1}): {type(e).__name__} - {e}")
            if attempt < API_MAX_RETRIES - 1:
                wait_time = API_RETRY_DELAY_SECONDS * (attempt + 1)
                print(f"Retrying in {wait_time} seconds...")
                await asyncio.sleep(wait_time)
            else:
                print(f"Max retries reached for {source_name}. SDK Error: {e}")
                return []
                
    print(f"All retries failed for {source_name} with SDK.")
    return []

## 5. Main Processing Orchestration: `main_extraction_process`

This function coordinates the entire extraction workflow:
-   Iterates through the defined newsletter HTML files.
-   Reads each file's content.
-   Calls `extract_metadata_with_gemini` to parse the HTML and extract data.
-   Aggregates all extracted data.
-   Converts the aggregated data into a Pandas DataFrame.
-   Includes a delay between processing each file.

In [15]:
async def main_extraction_process():
    """Main function to orchestrate the extraction process."""
    all_extracted_data = []

    if not SDK_CONFIGURED_SUCCESSFULLY:
         # Check if we are in Canvas and API_KEY is "", allowing a special case
        if not (IS_CANVAS_ENV and API_KEY == ""):
            print("Critical Error: Google Generative AI SDK is not configured, and not in a potential Canvas fallback scenario. Aborting extraction.")
            return pd.DataFrame()
        else:
            print("Warning: SDK not explicitly configured, relying on potential Canvas environment implicit configuration for the SDK (this might fail).")


    for i, newsletter_info in enumerate(newsletter_files_info):
        source_name = newsletter_info["name"]
        file_path = newsletter_info["path"]
        
        print(f"\n--- Processing newsletter: {source_name} ({file_path}) ---")
        html_content = read_html_file(file_path, source_name_for_dummy_content=source_name)
        
        if html_content:
            extracted_data_for_source = await extract_metadata_with_gemini(html_content, source_name)
            if isinstance(extracted_data_for_source, list) and extracted_data_for_source:
                 all_extracted_data.extend(extracted_data_for_source)
            else:
                print(f"No data extracted or an error occurred for {source_name}.")
            
            if i < len(newsletter_files_info) - 1: 
                print(f"Pausing for {API_CALL_DELAY_SECONDS} seconds before processing the next file...")
                await asyncio.sleep(API_CALL_DELAY_SECONDS)
        else:
            print(f"Skipping {source_name} due to issues reading the HTML file.")

    if not all_extracted_data:
        print("\nNo data was extracted from any newsletter after processing all files.")
        return pd.DataFrame()

    df = pd.DataFrame(all_extracted_data)
    desired_columns = [
        'newsletter_source', 'product_name', 'category', 'price', 
        'original_price', 'discount', 'promotion_description', 
        'availability', 'product_url', 'image_url'
    ]
    present_columns = [col for col in desired_columns if col in df.columns]
    missing_desired_cols = [col for col in desired_columns if col not in df.columns]
    if missing_desired_cols:
        print(f"\nNote: The following desired columns were not found in any extracted data and will be omitted: {missing_desired_cols}")
    
    extra_columns = [col for col in df.columns if col not in present_columns]
    final_columns_order = present_columns + extra_columns
    
    if not df.empty: # Ensure DataFrame is not empty before reordering
        df = df[final_columns_order]

    return df

## 6. Execute Extraction and View Results

The following cell runs the main extraction process.
- The extracted data will be compiled into a Pandas DataFrame.
- The DataFrame will be printed.
- The DataFrame will be saved as a CSV file.

**Note:** This process involves API calls and may take some time.

In [16]:
async def run_and_display_results():
    print("Starting the newsletter data extraction process using Google Generative AI SDK...")
    extracted_df = await main_extraction_process()

    if not extracted_df.empty:
        print("\n--- Extracted Product Metadata (DataFrame) ---")
        print(extracted_df.to_string())
        try:
            extracted_df.to_csv(OUTPUT_CSV_PATH, index=False, encoding='utf-8')
            print(f"\nSuccessfully saved extracted metadata to: {OUTPUT_CSV_PATH}")
        except Exception as e:
            print(f"\nError saving DataFrame to CSV at {OUTPUT_CSV_PATH}: {e}")
    else:
        print("\n--- No Product Metadata Extracted ---")
        print("The resulting DataFrame is empty.")

In [17]:
await run_and_display_results()

Starting the newsletter data extraction process using Google Generative AI SDK...

--- Processing newsletter: konvy (/mnt/c/Users/tduricic/Development/workspace/conversational-reco/data/raw/konvy.html) ---
Starting metadata extraction for konvy using Google Generative AI SDK...
Attempt 1 of 3 for konvy with SDK...
Successfully extracted 6 items for konvy using SDK.
Pausing for 7 seconds before processing the next file...

--- Processing newsletter: gramedia (/mnt/c/Users/tduricic/Development/workspace/conversational-reco/data/raw/gramedia.html) ---
Starting metadata extraction for gramedia using Google Generative AI SDK...
Attempt 1 of 3 for gramedia with SDK...
Successfully extracted 9 items for gramedia using SDK.
Pausing for 7 seconds before processing the next file...

--- Processing newsletter: uniqlo (/mnt/c/Users/tduricic/Development/workspace/conversational-reco/data/raw/uniqlo.html) ---
Starting metadata extraction for uniqlo using Google Generative AI SDK...
Attempt 1 of 3 fo

NameError: name 'extracted_df' is not defined